# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [10]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("D:\School\Challenge\Weather-Database\WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Current Description,Date
0,0,Hualmay,PE,-11.0964,-77.6139,74.62,83,100,8.32,overcast clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
1,1,Kaitangata,NZ,-46.2817,169.8464,60.17,56,5,13.98,clear sky,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
2,2,Norman Wells,CA,65.2820,-126.8329,12.24,73,20,5.75,few clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
3,3,Punta Arenas,CL,-53.1500,-70.9167,41.11,93,100,8.05,light intensity drizzle rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
4,4,Lebu,CL,-37.6167,-73.6500,51.51,90,9,12.53,clear sky,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [11]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [12]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Current Description,Date
16,16,Salalah,OM,17.0151,54.0924,84.29,70,40,14.97,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
17,17,Rikitea,PF,-23.1203,-134.9692,78.76,81,99,15.99,moderate rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
20,20,Bondo,CD,3.8146,23.6866,79.63,65,53,3.00,broken clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
22,22,Padang,ID,-0.9492,100.3543,87.85,70,35,4.85,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
24,24,Tuy Hoa,VN,13.0833,109.3000,85.05,65,34,9.46,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
30,30,Vaitape,PF,-16.5167,-151.7500,82.44,72,89,10.69,overcast clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
31,31,Avarua,CK,-21.2078,-159.7750,77.05,88,100,10.36,heavy intensity rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
40,40,Alofi,NU,-19.0595,-169.9187,78.69,94,100,2.30,light rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
43,43,Flinders,AU,-34.5833,150.8552,80.19,69,100,5.99,moderate rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
47,47,Carnarvon,AU,-24.8667,113.6333,78.93,55,100,12.41,overcast clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [13]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind,Current Description,Date
16,16,Salalah,OM,17.0151,54.0924,84.29,70,40,14.97,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
17,17,Rikitea,PF,-23.1203,-134.9692,78.76,81,99,15.99,moderate rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
20,20,Bondo,CD,3.8146,23.6866,79.63,65,53,3.00,broken clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
22,22,Padang,ID,-0.9492,100.3543,87.85,70,35,4.85,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
24,24,Tuy Hoa,VN,13.0833,109.3000,85.05,65,34,9.46,scattered clouds,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
...,...,...,...,...,...,...,...,...,...,...,...
680,680,Iranshahr,IR,27.2025,60.6848,77.34,25,0,6.91,clear sky,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
681,681,Bundaberg,AU,-24.8500,152.3500,83.21,68,8,3.00,clear sky,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
684,684,Micheweni,TZ,-4.9667,39.8333,84.65,67,66,3.31,light rain,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."
689,689,Mahibadhoo,MV,3.7833,72.9667,82.65,74,0,8.43,clear sky,"[{'City': 'Hualmay', 'Latitude': -11.0964, 'Lo..."


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Latitude", "Longitude"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [16]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
16,Salalah,OM,84.29,scattered clouds,17.0151,54.0924,
17,Rikitea,PF,78.76,moderate rain,-23.1203,-134.9692,
20,Bondo,CD,79.63,broken clouds,3.8146,23.6866,
22,Padang,ID,87.85,scattered clouds,-0.9492,100.3543,
24,Tuy Hoa,VN,85.05,scattered clouds,13.0833,109.3000,
30,Vaitape,PF,82.44,overcast clouds,-16.5167,-151.7500,
31,Avarua,CK,77.05,heavy intensity rain,-21.2078,-159.7750,
40,Alofi,NU,78.69,light rain,-19.0595,-169.9187,
43,Flinders,AU,80.19,moderate rain,-34.5833,150.8552,
47,Carnarvon,AU,78.93,overcast clouds,-24.8667,113.6333,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [17]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [22]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Salalah - nearest hotel: Muscat International Hotel
Rikitea - nearest hotel: Chez Bianca & Benoit
Bondo - nearest hotel: No hotel found
Padang - nearest hotel: losmen Surya
Tuy Hoa - nearest hotel: Vinh Tuan Hotel
Vaitape - nearest hotel: Oa Oa Lodge
Avarua - nearest hotel: Paradise Inn
Alofi - nearest hotel: No hotel found
Flinders - nearest hotel: No hotel found
Carnarvon - nearest hotel: No hotel found
Bachaquero - nearest hotel: No hotel found
Namibe - nearest hotel: Hotel Chik Chik Namibe
Arraial Do Cabo - nearest hotel: No hotel found
Matara - nearest hotel: Villa Palm Breeze
Vaini - nearest hotel: Keleti International Resort
Saint-Pierre - nearest hotel: Tropic Hotel
Mahebourg - nearest hotel: Grand Bel Air
Mackay - nearest hotel: Mackay Oceanside Central Apartment Hotel
Taoudenni - nearest hotel: No hotel found
Balkanabat - nearest hotel: Nebitçi
Mayumba - nearest hotel: Résidence Hôtelière Fabso
Sur - nearest hotel: Sur Hotel
Butaritari - nearest hotel: N

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
16,Salalah,OM,84.29,scattered clouds,17.0151,54.0924,Muscat International Hotel
17,Rikitea,PF,78.76,moderate rain,-23.1203,-134.9692,Chez Bianca & Benoit
20,Bondo,CD,79.63,broken clouds,3.8146,23.6866,No hotel found
22,Padang,ID,87.85,scattered clouds,-0.9492,100.3543,losmen Surya
24,Tuy Hoa,VN,85.05,scattered clouds,13.0833,109.3000,Vinh Tuan Hotel
...,...,...,...,...,...,...,...
680,Iranshahr,IR,77.34,clear sky,27.2025,60.6848,هتل قصِر
681,Bundaberg,AU,83.21,clear sky,-24.8500,152.3500,Young Australian Hotel
684,Micheweni,TZ,84.65,light rain,-4.9667,39.8333,No hotel found
689,Mahibadhoo,MV,82.65,clear sky,3.7833,72.9667,Coral View Inn


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [26]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No hotel found'].index, inplace = False)

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
16,Salalah,OM,84.29,scattered clouds,17.0151,54.0924,Muscat International Hotel
17,Rikitea,PF,78.76,moderate rain,-23.1203,-134.9692,Chez Bianca & Benoit
22,Padang,ID,87.85,scattered clouds,-0.9492,100.3543,losmen Surya
24,Tuy Hoa,VN,85.05,scattered clouds,13.0833,109.3000,Vinh Tuan Hotel
30,Vaitape,PF,82.44,overcast clouds,-16.5167,-151.7500,Oa Oa Lodge
...,...,...,...,...,...,...,...
670,Wewak,PG,84.74,overcast clouds,-3.5534,143.6268,Village Inn
680,Iranshahr,IR,77.34,clear sky,27.2025,60.6848,هتل قصِر
681,Bundaberg,AU,83.21,clear sky,-24.8500,152.3500,Young Australian Hotel
689,Mahibadhoo,MV,82.65,clear sky,3.7833,72.9667,Coral View Inn


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [27]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [34]:

# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Max Temp,Hotel Name,Country,Current Description)